<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/segmentation-to-CT_scan-simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2025 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Digital twin of a CT synchrotron beamline: [Dual Imaging And Diffraction (DIAD)](https://www.diamond.ac.uk/Instruments/Imaging-and-Microscopy/DIAD.html)

Summary here.
<!-- This notebook shows how to load a segmented image and you it to create a multi-part sample. 
It relies on functions built in gVXR, i.e. no third-party software is required. 
The functionality is multi-threaded to boost performances. 
It is then used to simulate a CT scan acquisition with gVXR and recontruct it with CIL. -->

![Illustration here]()

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the package with `!pip install gvxr`
</div>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !apt-get install libnvidia-gl-575
    !pip install -q condacolab
    import condacolab
    condacolab.install()

    !conda install -y -c conda-forge -c https://software.repos.intel.com/python/conda -c ccpi cil=24.3.0 ipp=2021.12 tigre

    !pip install gvxr

In [ ]:
# import os # Create the output directory if necessary
# import numpy as np # Who does not use Numpy?

# from tifffile import  imread
# import matplotlib.pyplot as plt # Plotting
import matplotlib.pyplot as plt

# from tqdm.notebook import tqdm

# User interface
import IPython
from IPython.display import display
from ipywidgets import widgets, Layout

#  CT simulation

# from gvxrPython3 import gvxr
# from gvxrPython3.gVXRDataReader import *

# # CT reconstruction
# from cil.recon import FDK # For CBCT
# from cil.recon import FBP # For parallel beam geometry

# from cil.io import TIFFWriter

# from cil.processors import TransmissionAbsorptionConverter
# from cil.utilities.display import show_geometry, show2D

Import the twin

In [ ]:
from gvxrPython3.twins.utils import createDigitalTwin

In [ ]:
twin = createDigitalTwin(name="DiaD")

In [ ]:
help(twin)

In [ ]:
twin.get_beams()

In [ ]:
twin.source

In [ ]:
twin.set_beam(twin.get_beams()[1])
print(twin.source)
print(twin.beam)

In [ ]:
help(twin.beam)

In [ ]:
twin.source.filters

Function to create a small UI

In [ ]:
def creatreUI():

    global twin

    twin_min_energy = 7
    twin_max_energy = 38

    # Select the spectrum to use
    # spectrum_type_widget = widgets.RadioButtons(
    #     options=['Monochromatic', 'Pink-beam (low angle)', 'Pink-beam (high angle)'],
    #     # description='Beam spectrum:',
    #     disabled=False,
    #     value='Monochromatic',
    #     layout={'width': 'max-content'}
    # )

    spectrum_type_widget = widgets.Select(
        options=twin.get_beams(),
        # description='Beam spectrum:',
        disabled=False,
        value=twin.get_beams()[0],
        layout={'width': 'max-content'}
    )

    # Make it look nice
    beam_selection_box = widgets.Box(
        [
            widgets.Label(value='Select the type of spectrum:'), 
            spectrum_type_widget
        ]
    )

    # If you selected a pink beam, i.e. a poychromatic beam as the monochromator was not used, you may select a filter.
    filter_material_type_widget = widgets.Select(
        options=['None', 'Al', 'Cu'],
        # description='Filter material:',
        disabled=False,
        value='None',
        layout={'width': 'max-content'}
    )

    # If you selected a pink beam, i.e. a poychromatic beam as the monochromator was not used, you may select a filter.
    filter_thickness_widget = widgets.Select(
        options=['0.5', '1.0'],
        # description='Filter thickness [mm]:',
        disabled=False,
        value='0.5',
        layout={'width': 'max-content'}
    )

    filter_selection_box = widgets.Box(
        [
            widgets.Label(value='Filter material:'), 
            filter_material_type_widget, 
            widgets.Label(value='Filter thickness [mm]:'), 
            filter_thickness_widget
        ]
    )

    # Else select the energy
    energy_selector_widget = widgets.FloatSlider(
        value=twin_min_energy + (twin_max_energy - twin_min_energy) / 2.0,
        min=twin_min_energy,
        max=twin_max_energy,
        step=0.1,
        # description='Mono-energy [keV]:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

    # Make it look nice
    mono_energy_box = widgets.Box(
        [
            widgets.Label(value='Mono-energy [keV]:'), 
            energy_selector_widget
        ]
    )


    if spectrum_type_widget.value == 'Monochromatic':
        filter_selection_box.disabled = True

        for child in filter_selection_box.children:
            child.disabled = True

        for child in mono_energy_box.children:
            child.disabled = False

    else:
        filter_selection_box.disabled = False

        for child in filter_selection_box.children:
            child.disabled = False

        for child in mono_energy_box.children:
            child.disabled = True
    
    # Select the exposure
    exposure_widget = widgets.FloatSlider(
        value=0.1,
        min=0.5,
        max=1.0,
        step=0.1,
        # description='Exposure [sec]:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )

    # Make it look nice
    exposure_box = widgets.Box(
        [
            widgets.Label(value='Exposure [sec]:'), 
            exposure_widget
        ]
    )

    # Plot the correspoding spectrum



    spectrum_plot_widget = widgets.Output(layout=Layout(width='50%'))


    def on_spectrum_type_value_change(change):
        # Update the UI
        if change['new'] == 'Monochromatic':
            filter_selection_box.disabled = True

            for child in filter_selection_box.children:
                child.disabled = True

            for child in mono_energy_box.children:
                child.disabled = False

        else:
            filter_selection_box.disabled = False

            for child in filter_selection_box.children:
                child.disabled = False

            for child in mono_energy_box.children:
                child.disabled = True

        # Plot the spectrum
        self.update_spectrum_plot()

    def on_filter_material_value_change(change):
        self.update_spectrum_plot()

    def on_energy_value_change(change):

        # Change the energy
        diad.beam.kev = change["new"]
        self.update_spectrum_plot()



    def update_spectrum_plot(self):
        with spectrum_plot_widget:
            # print(change["new"])

            IPython.display.clear_output(True)
            fig = plt.figure(figsize=(10,5), dpi=90)
            fig.canvas.toolbar_visible = False
            ax = fig.add_subplot(111)
            ax.plot([0,1,2], [0,exposure_widget.value,2])
            ax.set_xlabel("Incident energy [keV]")
            ax.set_ylabel("Number of photons")
            plt.show()

    # Update the visibility of widgets
    spectrum_type_widget.observe(on_spectrum_type_value_change, names='value')

    # Update the filter thicknesses based on the material
    filter_material_type_widget.observe(on_filter_material_value_change, names='value')

    # Update the spectrum
    spectrum_type_widget.observe(on_energy_value_change, names='value')
    energy_selector_widget.observe(on_energy_value_change, names='value')
    filter_material_type_widget.observe(on_energy_value_change, names='value')
    filter_thickness_widget.observe(on_energy_value_change, names='value')
    exposure_widget.observe(on_energy_value_change, names='value')


    layout_left_column = widgets.VBox([beam_selection_box, mono_energy_box, filter_selection_box, exposure_box])
    layout_two_columns = widgets.HBox([layout_left_column, spectrum_plot_widget])


    display(layout_two_columns)

creatreUI()

Instantiate the twin

In [ ]:
twin = None

Display the UI

In [ ]:
creatreUI(twin)

## Getting the data ready

Where to save the data.

In [ ]:
output_path = "../notebooks/output_data/DIAD"
if not os.path.exists(output_path):
    os.makedirs(output_path)

## 1. Set the simulation environment

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context");
gvxr.createOpenGLContext();

In [ ]:
rotation_centre = [0, 0, 0]

In [ ]:
# Create a source
print("Set up the beam")
gvxr.setSourcePosition(-40.0,  0.0, 0.0, "cm")
gvxr.usePointSource()
#  For a parallel source, use gvxr.useParallelBeam()

In [ ]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.addEnergyBinToSpectrumPerPixelAtSDD(33, "keV", 97)
gvxr.addEnergyBinToSpectrumPerPixelAtSDD(66, "keV", 2)
gvxr.addEnergyBinToSpectrumPerPixelAtSDD(99, "keV", 1)

In [ ]:
# Set up the detector
print("Set up the detector")
gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm")
gvxr.setDetectorUpVector(0, 0, 1)
gvxr.setDetectorNumberOfPixels(1000, 3)
gvxr.setDetectorPixelSize(1.0, 1.0, "um")
gvxr.setScintillator("GGG", 170, "um")

## 2. Create the sample from a segmented image

Read the image

In [ ]:
labels = imread("../data/labels.tif")
voxel_size = [1.5, 1.5, 1.5]
unit = "um"

Display the image

In [ ]:
plt.figure()
plt.imshow(labels,
    origin='upper',
    extent=[0.0, voxel_size[0] * labels.shape[1],
        0.0, voxel_size[1] * labels.shape[0]]
)
plt.xlabel("Pixel position [" + unit + "]")
plt.ylabel("Pixel position [" + unit + "]")
plt.show()

Make sure it is a 3D image

In [ ]:
if len(labels.shape) == 2:
    labels.shape = [1, *labels.shape]

Associate the labels with actual materials

In [ ]:
material_composition = {
    112: {
        'material type': 'element', 
        'material': 'C', 
        'density': 2.26
    }
    ,

    170: {
        'material type': 'compound', 
        'material': 'SiC', 
        'density': 3.21
    },

    202: {
        'material type': 'mixture', 
        'material': 'Ti90Al6V4', 
        'density': 4.43
    }
}

Process the segmentation to create iso surfaces

In [ ]:
for label in tqdm(np.unique(labels)):

    if label in material_composition.keys():
        selected_material = material_composition[label]
        mesh_label = material_composition[label]['material']

        print("Process", mesh_label)

        # Select the structure
        binary_image = (labels == label).astype(np.uint8)

        # Apply the Marching cubes
        gvxr.makeIsoSurface(mesh_label,
            binary_image,
            1,
            *rotation_centre,
            *voxel_size,
            "um"
        )

        # Save the mesh as an STL file
        gvxr.saveSTLfile(mesh_label, os.path.join(output_path, mesh_label+".stl"))

        # Set the material
        if selected_material['material type'].upper() == 'ELEMENT':
            print("\tUse element", mesh_label)
            gvxr.setElement(mesh_label, mesh_label)

            if "density" in selected_material:
                gvxr.setDensity(mesh_label, selected_material["density"], "g/cm3")

        elif selected_material['material type'].upper() == 'COMPOUND':
            print("\tUse compound", mesh_label)
            gvxr.setCompound(mesh_label, mesh_label)
            gvxr.setDensity(mesh_label, selected_material["density"], "g/cm3")
        
        elif selected_material['material type'].upper() == 'MIXTURE':
            print("\tUse mixture", mesh_label)
            gvxr.setMixture(mesh_label, mesh_label)
            gvxr.setDensity(mesh_label, selected_material["density"], "g/cm3")
        
        else:
            raise IOError("Invalid material type")

        # Add the material
        gvxr.addPolygonMeshAsInnerSurface(mesh_label)
    else:
        print(label, "is not in", material_composition)


Simulate the CT scan

In [ ]:
rotation_centre = [0, 0, 0]
number_of_projections = 3000

gvxr.computeCTAcquisition(
    os.path.join(output_path, "projections"), # The path where the X-ray projections will be saved
    "", # The path where the screenshots will be saved
    number_of_projections, # The total number of projections to simulate
    0, # The rotation angle corresponding to the first projection
    False, # A boolean flag to include or exclude the last angle
    360, # The rotation angle corresponding to the first projection
    50, # The number of white images used to perform the flat-field correction
    *rotation_centre, # The location of the rotation centre
    "mm", # The corresponding unit of length
    0, 0, 1 # The rotation axis
)  

Read the simulated data with CIL

In [ ]:
reader = gVXRDataReader(gvxr.getProjectionOutputPathCT(),
    gvxr.getAngleSetCT(),
    rotation_centre)

data = reader.read()

In [ ]:
print("data.geometry", data.geometry)

Apply the minus log transformation (use use white_level=1.0 as the flat-field correction is already applied)

In [ ]:
data_corr = TransmissionAbsorptionConverter(white_level=1.0)(data)

In [ ]:
data_corr.reorder(order='tigre')

We only want to reconstruct the slice in the middle of the volume

In [ ]:
ig = data_corr.geometry.get_ImageGeometry();

ig.voxel_num_z = 1
print("Image geometry", ig)

In [ ]:
# Perform the reconstruction with CIL
FDK_reconstruction = FDK(data_corr, ig).run()

Apply a circular mask as we simulated a region of interest scan

In [ ]:
FDK_reconstruction.apply_circular_mask()

Save the reconstructed CT images

In [ ]:
writer = TIFFWriter(data=FDK_reconstruction, file_name=os.path.join(output_path, "recons-" + str(number_of_projections), "slice_"), compression="uint16");
writer.write();

Show the CT slice

In [ ]:
show2D(FDK_reconstruction)

# Cleaning up

Once we have finished, it is good practice to clean up the OpenGL contexts and windows with the following command. Note that due to the object-oriented programming nature of the core API of gVXR, this step is automatic anyway.

In [ ]:
gvxr.destroy()